# LoopTree Tutorial

LoopTree is a model to evaluate the latency and energy of a fused-layer dataflow accelerator.

To model energy and latency, a workload, architecture, and mapping have to be specified. First, we discuss how these are specified. Then, we show how to run the LoopTree model.

## Specifying Architecture, Workload, and Mapping

For the LoopTree model to estimate energy and latency, the user must specify an architecture, workload, and mapping. Below, we discuss how to specify each of these inputs.

### Architecture
In LoopTree, the architecture of an accelerator is abstracted as buffers that use the Buffet semantics and computation units, following the [Timeloop v4 specification](https://timeloop.csail.mit.edu/v4).

An example architecture that we will use here is displayed below.

In [1]:
from util import *
from pprint import pp
show_config('architecture.yaml')

architecture:
  version: 0.4
  nodes:

  - !Component
    name: MainMemory
    class: DRAM
    attributes: {width: 256, block_size: 32, word_bits: 8, datawidth: 8}
    required_actions: ['read', 'write']



  - !Component
    name: IntermediateBuffer
    class: SRAM
    attributes:
      depth: 8192
      width: 64
      block_size: 32
      word_bits: 8
      datawidth: 8
  
  # - !Container
  #   name: system_arch
  #   attributes:
  #     # Top-level attributes inherited by all components unless overridden
  #     technology: "45nm"
  #     global_cycle_seconds: 1e-9
  #     datawidth: 16
    

  - !Component
    name: DRAM # offchip DRAM is the source of all datatypes
    class: DRAM # assume DRAM is large enough to store all the data, so no depth specification needed
    attributes:
      width: 64 # width in bits
      datawidth: 16
    required_actions: ['read', 'write']

  # - !Container
  #   name: chip

  # - !Container
  #   name: NNengine
  #   spatial: { meshX: 16, meshY: 

### Workloads

DNN workloads in LoopTree are abstracted as a set of Einsums, each representing a layer.

Each Einsum is specified as a dictionary. Each Einsum reads/writes to a number of tensors (referred to as `data_spaces` in the specification), and each tensor is either an input to the operation or an output (indicated by a key "read_write" with value `True`). Moreover, a "projection" from the Einsum to each tensor, which describes the index of the element in the data that an operation in the Einsum accesses, must be specified. Finally, we specify must specify the *shape* of the Einsum, which is the bounds of its dimensions (also referred to as "ranks").

An example comprising two fully-connected layers are shown below.

In [2]:
# show_config('two_fc.workload.yaml')

In this workload specification, we specify two Einsums Fc1 and Fc2. Each Einsum has three dimensions. Fc1 has dimensions P1, M1, and C1; Fc2 has dimensions P2, M2, and C2. Finally, we specify the shape of the Einsums as bounds for each Einsum dimension (rank).

### Mapping

The LoopTree mapping is a tree-structure that contains nodes of the types described below.
- **Loops**: a loop node specifies a rank in the Einsum that is partitioned and the shape of the tiles that results from the partitioning. If the loop is a "temporal" loop, then the tiles are scheduled to be processed one at a time. If the loop is a "spatial" loop, then the tiles are scheduled to parallel hardware units.
- **Branching point**: nodes above a branching point (*i.e.*, ancestor nodes) describe inter-Einsum mapping, which is applied to all Einsums under that branching point. Nodes underneath the branching point (*i.e.*, within each branch) pertain only to the Einsum of that branch.
- **Storage**: a storage node specifies the tiles of tensors to retain and which hardware level retains the tiles.
- **Compute**: a compute node specifies the hardware level used to compute operations of an Einsum. This node has to be a leaf node, and it also denotes the Einsum that a particular branch pertains to.

First, we discuss a layer-by-layer mapping example.

In [3]:
# show_config('layer-by-layer.mapping.yaml')

Explanation:
- (1.a) The storage node specifies that the tensors specified by the `dspace` key will be retained in target 0, which we will bind to MainMemory.
        Note that MainMemory retains Fmap2.
- (1.b) The sequential node specifies that the following branches (one for Fc1 and one for Fc2, as we will see shortly) are processed sequentially.
- (1.c) Filter1 is retained in GlobalBuffer
- (1.d) The rank P1 (which is a rank of the Fc1 Einsum) is partitioned into tiles with shape 1 and we will iterate over the tiles one at a time (temporal iteration).
- (1.e) Tiles of Fmap1 and Fmap2 are retained in GlobalBuffer. We retain tiles becuase the P1 rank has been partitioned.
- (1.f) Operations of Fc1 are processed in the MACC unit. Moreover, this node specifies that this branch is relevant to Fc2.
- (1.g) This and the following nodes specify how Fc2 is mapped.

In [4]:
# show_config('fused.mapping.yaml')

Explanation:
- (2.a) Similar to (1.a), but note that Fmap2 is no longer retained in MainMemory.
- (2.b) In this mapping, the $P2$ rank is partitioned to create tiles that are rows of feature maps Fmap1, Fmap2, and Fmap3. The tile shape attribute of the loop node implies that the tiles of Fmap3 have shape 1 in the $P2$ rank. LoopTree infers the shape of the tiles of Fmap2 and Fmap1 based on what is required as inputs to compute the specified tile of Fmap3. In this case, tiles that each contains one row of Fmap2 is required to compute tiles of Fmap3, and tiles that each contains one row of Fmap1 is required to compute tiles of Fmap2 in turn.
- (2.c) The tiles of Fmap1, Fmap2, and Fmap3 are retained in GlobalBuffer.
- (2.d) Similar to before, Fc1 and Fc2 are processed sequentially. However, as we have put node (2.b) above this sequential node, *tiles* of Fc1 and Fc2 are processed sequentially in alternating fashion: a tile of Fc1 produces a tile of Fmap2, the tile of Fmap2 is consumed by a tile of Fc2, another tile of Fc1 is produced, and so on.

## Running the Model

Running with the four matrix multiplications in a row (Test 1)

In [25]:
from pytimeloop.looptree.run import run_looptree
from util import *
from pprint import pp
# show_config('architecture.yaml')
# As previously mentioned, bindings map levels specified in the mapping
# to the hardware units specified in the architecture spec.
bindings = {
    0: 'MainMemory',
    1: 'IntermediateBuffer',
    # 2: 'LocalBuffer',
    # 3: 'input_reg',
    # 4: 'weight_reg',
    # 5: 'output_reg',
    3: 'MACC'
}

stats = run_looptree(
    CONFIG_DIR,
    # ['architecture.yaml', 'two_fc.workload.yaml', 'layer-by-layer.mapping.yaml'],
    ['architecture.yaml', 'matrix_multiplication_workload.yaml','matrix_mapping.yaml'],
    TMP_DIR,
    bindings,
    call_accelergy=True
)

In [23]:
print('Latency:', stats.latency)

Latency: 8388608


and also energy, which is computed by calculating the number of actions to each hardware component and multiplying that with the energy per action estimated using the Accelergy tool:

In [24]:
print('Energy:')
pp(stats.energy)

Energy:
{('MainMemory', 'read'): 167772160.0,
 ('IntermediateBuffer', 'read'): 3054177943.552,
 ('IntermediateBuffer', 'write'): 974974713.8560001,
 ('MainMemory', 'write'): 67108864.0,
 ('MACC', 'compute'): 7088373.76}


In [1]:
### with CONV files
from util import *
from pprint import pp
from pytimeloop.looptree.run import run_looptree

# As previously mentioned, bindings map levels specified in the mapping
# to the hardware units specified in the architecture spec.
bindings = {
    0: 'MainMemory',
    1: 'IntermediateBuffer',
    # 2: 'LocalBuffer',
    # 3: 'input_reg',
    # 4: 'weight_reg',
    # 5: 'output_reg',
    3: 'MACC',
    4: 'input_reg',
    5: 'weights_reg',
    6: 'output_reg',
    # 7: 'PE'
}

stats = run_looptree(
    CONFIG_DIR,
    # ['architecture.yaml', 'two_fc.workload.yaml', 'layer-by-layer.mapping.yaml'],
    ['architecture.yaml', 'conv_workload.yaml','conv_mapping.yaml'],
    TMP_DIR,
    bindings,
    call_accelergy=True
)

ParseError: Error calling cast function "Architecture" for value "{'version': 0.4, 'nodes': [{'name': 'MainMemory', 'class': 'DRAM', 'attributes': {'width': 256, 'block_size': 32, 'word_bits': 8, 'datawidth': 8}, 'required_actions': ['read', 'write']}, {'name': 'IntermediateBuffer', 'class': 'SRAM', 'attributes': {'depth': 8192, 'width': 256, 'block_size': 32, 'word_bits': 8, 'datawidth': 8}, 'required_actions': ['read', 'write']}, {'name': 'nnEngine', 'spatial': {'meshX': 4, 'meshY': 4}, 'attributes': {'depth': 16, 'width': 16, 'datawidth': 8}}, {'name': 'PE', 'spatial': {'meshX': 8, 'meshY': 8}, 'attributes': {'depth': 8, 'width': 8, 'datawidth': 8}}, {'name': 'scratchpad', 'class': 'smart_storage', 'attributes': {'depth': 128, 'datawidth': 8, 'width': 8}}, {'name': 'input_reg', 'class': 'regfile', 'attributes': {'depth': 16, 'datawidth': 8, 'width': 8}, 'constraints': {'dataspace': {'keep': ['Input', 'OutputVector1', 'OutputVector2', 'OutputVector3'], 'bypass': ['Matrix_1', 'Matrix_

In [ ]:
print('Latency:', stats.latency)
print('Energy:')
pp(stats.energy)

Now, we compare these results with the fused mapping:

In [1]:
from pytimeloop.looptree.run import run_looptree

# As previously mentioned, bindings map levels specified in the mapping
# to the hardware units specified in the architecture spec.
bindings = {
    0: 'MainMemory',
    1: 'GlobalBuffer',
    2: 'MACC'
}

stats = run_looptree(
    CONFIG_DIR,
    ['architecture.yaml', 'two_fc.workload.yaml', 'fused.mapping.yaml'],
    TMP_DIR,
    bindings,
    call_accelergy=True
)
print('Latency:', stats.latency)
print('Energy:')
pp(stats.energy)

NameError: name 'CONFIG_DIR' is not defined

As we can see, the energy consumption due to MainMemory reads and writes have decreased significantly from fusion.

Here, we have not modeled the impact of limited MainMemory bandwidth; thus, because the MACC unit utilization is the same (100%) in both cases, the latency is the same. However, if the layer-by-layer latency is limited by MainMemory bandwidth, fusion will also decrease latency.

In [19]:
from pytimeloop.looptree.run import run_looptree

# As previously mentioned, bindings map levels specified in the mapping
# to the hardware units specified in the architecture spec.
bindings = {
    0: 'MainMemory',
    1: 'GlobalBuffer',
    2: 'MACC'
}

stats = run_looptree(
    CONFIG_DIR,
    ['architecture.yaml', 'two_fc.workload.yaml', 'fused-pipeline.mapping.yaml'],
    TMP_DIR,
    bindings,
    call_accelergy=True
)
print('Latency:', stats.latency)
print('Energy:')
pp(stats.energy)

Latency: 528384
Energy:
{('MainMemory', 'read'): 50331648.0,
 ('GlobalBuffer', 'read'): 382769037.31200004,
 ('GlobalBuffer', 'write'): 123521941.50400001,
 ('MainMemory', 'write'): 16777216.0,
 ('MACC', 'compute'): 886046.72}
